<a href="https://colab.research.google.com/github/Steviey/DarwinexLabs/blob/master/HybridMult_NEW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#%matplotlib inline
import warnings, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
warnings.simplefilter(action='ignore', category=FutureWarning)

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Psl-Info: Using Google CoLab")
except:
    print("Psl-Info: Not using Google CoLab")
    COLAB = False

if COLAB:

    if not os.path.ismount('/content/drive'):
        drive.mount('/content/drive', force_remount=False)
    else:
        print("Psl-Info: Google Drive is already mounted.")

import importlib
import subprocess
import sys

def install_if_needed(package_name):
    try:
        importlib.import_module(package_name)
        print(f"Psl-Info: {package_name} is already installed.")
    except ImportError:
        print(f"Psl-Info: {package_name} is not installed. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

for package in ['tensorflow', 'numpy', 'cmaes','keras-tcn', 'keras-tuner', 'pandas', 'pyreadr', 'optuna']:
    install_if_needed(package)

class PathManager:
    def __init__(self, base_path):
        self.base_path = base_path

    def get_script_path(self, file_name):
        return f"{self.base_path}/{file_name}"

if not COLAB:
    path_manager = PathManager('/home/rstud/R/adthocStrat')
else:
    path_manager = PathManager('/content/drive/My Drive/ColabData')

import pyreadr
import pandas as pd
import numpy as np
import optuna

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # Set to ERROR to only show errors
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, SimpleRNN, Conv1D, Layer, Input
from tensorflow.keras import backend as K
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention

train_data = pyreadr.read_r(path_manager.get_script_path('python_train_data.Rds'))[None]
test_data = pyreadr.read_r(path_manager.get_script_path('python_test_data.Rds'))[None]
setup_data = pyreadr.read_r(path_manager.get_script_path('python_setup_data.Rds'))[None]
all_data = pyreadr.read_r(path_manager.get_script_path('python_all_data.Rds'))[None]
future_data = pyreadr.read_r(path_manager.get_script_path('python_future_data.Rds'))[None]
validation_dataP = pyreadr.read_r(path_manager.get_script_path('python_validation_data.Rds'))[None]
# from IPython.display import Image

# Image(filename='neddwplot (5).png')  # Replace with the actual path/filename

# ![My Image](neddwplot (5).png)

setupList = {
    'exogNames': setup_data['exogNames'].tolist(),
    'trainSetup': setup_data['trainSetup'][0],
    'time_steps': setup_data['time_steps'][0],
    'qRank': setup_data['qRank'][0],
    'part': setup_data['part'][0],
    'epochs': setup_data['epochs'][0],
    'nn_tuner': setup_data['nn_tuner'][0],
    'max_trials': setup_data['max_trials'][0],
    'patience': setup_data['patience'][0],
    'validation_split': setup_data['validation_split'][0],
    'executions_per_trial': setup_data['executions_per_trial'][0],
    'algoSrc': setup_data['algoSrc'][0],
    'layer_size': setup_data['layer_size'][0],
    'filters': setup_data['filters'][0],
    'ff_dim': setup_data['ff_dim'][0],
    'lot_type': setup_data['lot_type'][0],
    'editDate': setup_data['editDate'][0],
    'sampler': setup_data['sampler'][0],
    'dropout_rate': setup_data['dropout_rate'][0],
    'kernel_size': setup_data['kernel_size'][0]

}

# print(train_data.columns)
# print(setupList['exogNames'])
# sys.exit()
# os._exit()

#def prepare_input(data, time_steps, exog_names):

    #data['ds']  = pd.to_datetime(data['ds'])
    #data['ds'] = pd.to_datetime(data['ds']).apply(lambda date: date.toordinal())


    # X = []
    # Y = []
    # for i in range(len(data) - time_steps):
    #     X.append(data.iloc[i:(i + time_steps)][exog_names].values)
    #     Y.append(data['y'].iloc[i + time_steps])
    # return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)


def prepare_input(data, time_steps, exog_names):
    # ... (previous code) ...
    data['ds']  = pd.to_datetime(data['ds'])

    # Filter only numerical columns from exog_names
    numerical_exog_names = data[exog_names].select_dtypes(include=np.number).columns.tolist()

    X = []
    Y = []
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:(i + time_steps)][numerical_exog_names].values)
        Y.append(data['y'].iloc[i + time_steps])
    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)



class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]), initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],), initializer="zeros", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Dense(ff_dim, activation="relu")
        self.ffn_output = Dense(embed_dim)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.ffn_output(ffn_output)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

class Time2Vec(tf.keras.layers.Layer):
    def __init__(self, k, **kwargs):
        super(Time2Vec, self).__init__(**kwargs)
        self.k = k

    def build(self, input_shape):
        self.w = self.add_weight(name='w', shape=(input_shape[1],),
                                 initializer='uniform', trainable=True)
        self.b = self.add_weight(name='b', shape=(input_shape[1],),
                                 initializer='uniform', trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs):
        time_linear = self.w * inputs + self.b
        time_periodic = tf.math.sin(tf.multiply(inputs, self.k))
        return tf.concat([time_linear, time_periodic], axis=-1)

    def get_config(self):
        config = super().get_config().copy()
        config.update({'k': self.k})
        return config

class HybridHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def positional_encoding(position, d_model):
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])


    def build(self, hp):
        inputs = Input(shape=self.input_shape)

        layer_type = hp.Choice('layer_type', ['Conv1D', 'LSTM', 'GRU', 'SimpleRNN','Transformer','Time2Vec','TransformerPos'])

        if layer_type == 'Conv1D':
            x = Conv1D(filters=hp.Int('filters', 32, 128),
                       kernel_size=hp.Int('kernel_size', 2, setupList['kernel_size']),
                       activation='relu')(inputs)

        elif layer_type == 'LSTM':
            x = LSTM(hp.Int('layer_size',32,setupList['layer_size']), return_sequences=True)(inputs)

        elif layer_type == 'GRU':
            x = GRU(hp.Int('layer_size', 32,setupList['layer_size']), return_sequences=True)(inputs)

        elif layer_type == 'SimpleRNN':
            x = SimpleRNN(hp.Int('layer_size', 32,setupList['layer_size']), return_sequences=True)(inputs)

        elif layer_type == 'Transformer':
            embed_dim = self.input_shape[-1]
            num_heads = hp.Int('num_heads', 2, 8)
            ff_dim = hp.Int('ff_dim', 32, setupList['ff_dim'])
            x = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)(inputs)

        elif layer_type == 'Time2Vec':
            #time_embedding = Time2Vec(input_dim=self.input_shape[1])(inputs)
            #x = tf.keras.layers.Concatenate()([inputs, time_embedding])
            k = hp.Int('time2vec_k', min_value=1, max_value=10, step=1)  # Hyperparameter for k
            x = Time2Vec(k)(inputs)  # Use the custom Time2Vec layer

        elif layer_type == 'TransformerPos':
            embed_dim = self.input_shape[-1]  # Get the embedding dimension
            num_heads = hp.Int('num_heads', 2, 8)
            ff_dim = hp.Int('ff_dim', 32, 256)

            # Calculate and apply positional encodings
            pos_encoding = positional_encoding(self.input_shape[0], embed_dim) #Assuming input_shape=(time_steps, features)
            # Add positional encodings to the input
            encoded_input = inputs + pos_encoding

            x = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)(encoded_input)

        x = Attention()(x)
        #x = Dense(hp.Int('layer_size', 32, 256), activation='relu')(x)
        x = Dense(hp.Int('layer_size', 32, setupList['layer_size']), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2_reg', 1e-5, 1e-2, sampling='LOG')))(x)
        x = Dropout(rate=hp.Float('dropout_rate', 0.1, setupList['dropout_rate']))(x)
        outputs = Dense(1)(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
        return model

def optimize_hyperparameters(X_train, Y_train, input_shape):
    hypermodel = HybridHyperModel(input_shape)  # Instantiate hypermodel

    if setupList['nn_tuner'] == 'optuna':
        def objective(trial):
            hp = {  # Manually create hp object from Optuna trial
                'layer_type': trial.suggest_categorical('layer_type', ['Conv1D', 'LSTM', 'GRU', 'SimpleRNN', 'Transformer']),
                'filters': trial.suggest_int('filters', 32, setupList['filters']),
                'kernel_size': trial.suggest_int('kernel_size', 2, setupList['kernel_size']),
                'layer_size': trial.suggest_int('layer_size', 32, setupList['layer_size']),
                'num_heads': trial.suggest_int('num_heads', 2, 8),
                'ff_dim': trial.suggest_int('ff_dim', 32, setupList['ff_dim']),
                'dropout_rate': trial.suggest_float('dropout_rate', 0.1, setupList['dropout_rate']),
            }

            # This part may need adjustment if HybridHyperModel expects an hp object from keras tuner
            # Convert Optuna's trial object to a Keras Tuner HyperParameters object
            # This is a workaround and may require you to adjust the structure of the hp dictionary
            from keras_tuner.engine import hyperparameters as hp_module
            hp_keras = hp_module.HyperParameters()
            for k, v in hp.items():
                hp_keras.Fixed(k, v)

            model = hypermodel.build(hp_keras)  # Pass the hp object to hypermodel.build

            early_stopping = EarlyStopping(monitor='val_loss', patience=setupList['patience'])

            # history = model.fit(X_train, Y_train, epochs=setupList['epochs'], batch_size=32,
            #                     validation_split=setupList['validation_split'], callbacks=[early_stopping], verbose=0)

            history = model.fit(X_train, Y_train, epochs=setupList['epochs'], batch_size=32,
                                validation_data=validation_dataP, callbacks=[early_stopping], verbose=0)

            return history.history['val_loss'][-1]

        ############################################################
        print('nax_trials:')
        print(setupList['max_trials'])

        print(type(setupList['max_trials']))

        print(f"Optuna version: {optuna.__version__}")

        #optuna.logging.set_verbosity(logging.WARNING)

        #,verbosity=optuna.logging.WARNING

        if setupList['max_trials'] >= 1000 or setupList['sampler']=='CmaEsSampler':
            study  = optuna.create_study(direction='minimize',sampler=optuna.samplers.CmaEsSampler(warn_independent_sampling=False))
        else:
            study = optuna.create_study(direction='minimize')

        ################
        # https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html
        ################
        # import optuna
        # from optuna.samplers import TPESampler

        # def objective(trial):
        #     x = trial.suggest_uniform('x', -10, 10)
        #     return x**2

        # study = optuna.create_study(sampler=TPESampler())

        # def objective(trial):
        #     x = trial.suggest_uniform('x', -100, 100)
        #     y = trial.suggest_int('y', -100, 100)
        #     return x ** 2 + y ** 2

        # search_space = {
        #     'x': [-50, 0, 50],
        #     'y': [-99, 0, 99]
        # }
        #study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space))
        ############################################################

        optuna.logging.set_verbosity(optuna.logging.WARNING)

        #study.optimize(objective, n_trials=3*3)

        study.optimize(objective, n_trials=setupList['max_trials'])

        if COLAB:
            from IPython.display import display
            display(optuna.visualization.plot_optimization_history(study))
            display(optuna.visualization.plot_param_importances(study))
            #display(optuna.visualization.plot_countour(study))


        best_trial = study.best_trial
        # Extract best hyperparameters from Optuna study
        best_hp = best_trial.params

        # This part requires constructing an hp object similar to what is expected by HybridHyperModel
        from keras_tuner.engine import hyperparameters as hp_module
        best_hp_keras = hp_module.HyperParameters()
        for k, v in best_hp.items():
            best_hp_keras.Fixed(k, v)

        best_model = hypermodel.build(best_hp_keras)  # Build the best model using best_hp_keras

    elif setupList['nn_tuner'] == 'keras-tuner':
        tuner = RandomSearch(
            hypermodel,  # Directly use hypermodel instance
            objective='val_loss',
            max_trials=setupList['max_trials'],
            executions_per_trial=setupList['executions_per_trial'],
            directory='hyperparameter_search',
            project_name='hybrid_model_tuning'
        )
        early_stopping = EarlyStopping(monitor='val_loss', patience=setupList['patience'])
        tuner.search(X_train, Y_train, epochs=setupList['epochs'], validation_data=validation_dataP, callbacks=[early_stopping])
        best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
        best_model = tuner.hypermodel.build(best_hp)  # Build the best model using best_hp from Keras Tuner
    else:
        raise ValueError(f"Unknown nn_tuner: {setupList['nn_tuner']}")

    return best_model, best_hp

#setupList['trainSetup']='prodPred'
#setupList['trainSetup']='prodTrain'

    print('\n')
    print(setupList['trainSetup'])
    print('\n')

if setupList['trainSetup'] == 'prodTrain':
    X_train, Y_train = prepare_input(train_data, time_steps=setupList['time_steps'], exog_names=setupList['exogNames'])
    X_test, Y_test = prepare_input(test_data, time_steps=setupList['time_steps'], exog_names=setupList['exogNames'])
    input_shape = (X_train.shape[1], X_train.shape[2])

    best_model, best_hp = optimize_hyperparameters(X_train, Y_train, input_shape)
    history = best_model.fit(X_train, Y_train, epochs=setupList['epochs'], validation_split=setupList['validation_split'])
    #history = best_model.fit(X_train, Y_train, epochs=setupList['epochs'])

    loss_df = pd.DataFrame({
        'epoch': range(1, len(history.history['loss']) + 1),
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss']
    })
    loss_df.to_csv(path_manager.get_script_path("python_training.csv"), index=False)

    predictions = best_model.predict(X_test)

    train_mae = history.history['mae'][-1]
    out_of_sample_predictions = predictions

    # # Calculate MAE
    #train_mae = np.mean(np.abs(Y_train - train_predictions.flatten()))
    out_of_sample_mae = np.mean(np.abs(Y_test - out_of_sample_predictions.flatten()))

    # Save best parameters to a CSV file
    best_params = best_hp if isinstance(best_hp, dict) else best_hp.values
    best_params['editDate'] = setupList['editDate']
    best_params['algoSrc'] = setupList['algoSrc']
    best_params['qRank'] = setupList['qRank']
    best_params['part'] = setupList['part']
    best_params['lot_type'] = setupList['lot_type']
    best_params['train_mae'] = train_mae
    best_params['out_of_sample_mae'] = out_of_sample_mae
    best_params_df = pd.DataFrame([best_params])

    csv_path = path_manager.get_script_path('best_parameters.csv')
    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
        updated_df = pd.concat([existing_df, best_params_df], ignore_index=True)
    else:
        updated_df = best_params_df

    updated_df.to_csv(csv_path, index=False)
    print(f"Best parameters have been appended to '{csv_path}'")

    #print(f"\nTrain MAE: {train_mae}")
    print(f"Out-of-Sample MAE: {out_of_sample_mae}")

    if COLAB:
        plt.figure(figsize=(12, 6))
        plt.plot(Y_test, label='True')
        plt.plot(predictions, label='Predicted')
        plt.xlabel('Time')
        plt.ylabel('Values')
        plt.title('Out-of-Sample Data: True vs Predicted')
        plt.legend()
        plt.show()

        # Training loss plot
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.show()

    print(setupList['nn_tuner'])

elif setupList['trainSetup'] == 'prodPred':
    best_parameters = pd.read_csv(path_manager.get_script_path("best_parameters.csv"))
    filtered_parameters = best_parameters[(best_parameters['algoSrc'] == setupList['algoSrc']) &
                                          (best_parameters['part'] == setupList['part']) &
                                          (best_parameters['lot_type'] == setupList['lot_type']) &
                                          (best_parameters['qRank'] == setupList['qRank'])]
    best_params = filtered_parameters.sort_values(by="out_of_sample_mae").iloc[0]
    input_shape = (setupList['time_steps'], len(setupList['exogNames']))

    # Access hyperparameter values directly
    layer_type = best_params['layer_type']
    filters = int(best_params['filters']) if 'filters' in best_params else None  # Handle optional hyperparameters
    kernel_size = int(best_params['kernel_size']) if 'kernel_size' in best_params else None
    layer_size = int(best_params['layer_size']) if 'layer_size' in best_params else None
    num_heads = int(best_params['num_heads']) if 'num_heads' in best_params else None
    ff_dim = int(best_params['ff_dim']) if 'ff_dim' in best_params else None
    dropout_rate = best_params['dropout_rate'] if 'dropout_rate' in best_params else None

    # Build the model manually using loaded hyperparameters
    inputs = Input(shape=input_shape)
    if layer_type == 'Conv1D':
        x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(inputs)
    elif layer_type == 'LSTM':
        x = LSTM(layer_size, return_sequences=True)(inputs)
    elif layer_type == 'GRU':
        x = GRU(layer_size, return_sequences=True)(inputs)
    elif layer_type == 'SimpleRNN':
        x = SimpleRNN(layer_size, return_sequences=True)(inputs)
    elif layer_type == 'Transformer':
        embed_dim = input_shape[-1]
        x = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)(inputs)
    else:
        raise ValueError(f"Unknown layer_type: {layer_type}")

    x = Attention()(x)
    x = Dense(layer_size, activation='relu')(x)
    if dropout_rate is not None:
        x = Dropout(rate=dropout_rate)(x)
    outputs = Dense(1)(x)

    best_model = Model(inputs=inputs, outputs=outputs)
    best_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    X_all, _ = prepare_input(all_data, time_steps=setupList['time_steps'], exog_names=setupList['exogNames'])
    predictions = best_model.predict(X_all[-1].reshape(1, setupList['time_steps'], -1))
    print("Next predicted value:", predictions[0][0])

Psl-Info: Using Google CoLab
Psl-Info: Google Drive is already mounted.
Psl-Info: tensorflow is already installed.
Psl-Info: numpy is already installed.
Psl-Info: cmaes is already installed.
Psl-Info: keras-tcn is not installed. Installing...
Psl-Info: keras-tuner is not installed. Installing...
Psl-Info: pandas is already installed.
Psl-Info: pyreadr is already installed.
Psl-Info: optuna is already installed.
nax_trials:
10
<class 'numpy.int32'>
Optuna version: 4.0.0


[W 2024-11-01 16:25:11,138] Trial 0 failed with parameters: {'layer_type': 'SimpleRNN', 'filters': 104, 'kernel_size': 3, 'layer_size': 37, 'num_heads': 5, 'ff_dim': 49, 'dropout_rate': 0.3346115708152877} because of the following error: ValueError("could not convert string to float: '2001-10-05 21:00:00'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-3-77a39759d2e0>", line 286, in objective
    history = model.fit(X_train, Y_train, epochs=setupList['epochs'], batch_size=32,
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/optree/ops.py", line 747, in tree_map
    return treespec.unflatten(map(func, *flat_args))
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py"

ValueError: could not convert string to float: '2001-10-05 21:00:00'